In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.tree import export_graphviz,plot_tree
from six import StringIO
from IPython.display import Image  
import pydotplus
import collections

In [4]:
df_final = pd.read_csv('./data/datosprocesados1.csv', delimiter=',', decimal='.', encoding='UTF-8')
df_final.head()

,IDDIST,cluster,res_platform,score,platform,polarity,num_score
0,14.0,4.0,2,5.0,2,5,0.8919
1,14.0,4.0,2,5.0,2,5,0.8781
2,14.0,4.0,2,1.0,2,1,0.9713
3,13.0,5.0,1,4.0,1,3,0.5874
4,13.0,5.0,1,5.0,1,5,0.5196


In [5]:
valores_score = df_final['score'].value_counts()
print(valores_score)

score
5.0    389655
4.0    210019
3.0     90142
1.0     37463
2.0     27787
Name: count, dtype: int64


In [6]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755066 entries, 0 to 755065
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   IDDIST        701056 non-null  float64
 1   cluster       752017 non-null  float64
 2   res_platform  755066 non-null  int64  
 3   score         755066 non-null  float64
 4   platform      755066 non-null  int64  
 5   polarity      755066 non-null  int64  
 6   num_score     755066 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 40.3 MB


In [7]:
df_final = df_final.dropna(subset=['IDDIST', 'cluster'])

In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 698143 entries, 0 to 755065
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   IDDIST        698143 non-null  float64
 1   cluster       698143 non-null  float64
 2   res_platform  698143 non-null  int64  
 3   score         698143 non-null  float64
 4   platform      698143 non-null  int64  
 5   polarity      698143 non-null  int64  
 6   num_score     698143 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 42.6 MB


In [9]:
X = df_final.drop('score', axis=1)  
y = df_final['score']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

column_names = X_train.columns.tolist()

# Escalar
scaler = StandardScaler()

X_train_esc = scaler.fit_transform(X_train)
X_train = DataFrame(X_train_esc, columns=column_names)

X_test_esc = scaler.fit_transform(X_test)
X_test = DataFrame(X_test_esc, columns=column_names)

print ("scaler")

scaler


In [8]:
tree = DecisionTreeClassifier(criterion="entropy", random_state=0)
tree = tree.fit(X_train,y_train)

print('Accuracy para datos de entrenamiento: {:.3f}'.format(tree.score(X_train, y_train)))
print('Accuracy para datos de test: {:.3f}'.format(tree.score(X_test, y_test)))

Accuracy para datos de entrenamiento: 0.878
Accuracy para datos de test: 0.498


In [ ]:
plot_tree(tree)

In [9]:
# numero de árboles: 5, 10, 20, 50 y 100
from sklearn.ensemble import RandomForestClassifier
#   []
n_arboles = {5, 10, 20, 50, 100}
particion = 'gini'
max_depth = 10
min_samples_split = 10 
min_samples_leaf = 2
max_feature = 'auto'

randomForest = [RandomForestClassifier(n_estimators = i, criterion= particion,
                                       max_depth = max_depth, min_samples_split = min_samples_split,
                                      min_samples_leaf = min_samples_leaf, max_features =None) for i in n_arboles]

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Entrenar el modelo
dicc_forest = {}
dicc_reporte = {}

for i, model in enumerate(randomForest):
    # Entrenar el modelo con datos de entrenamiento
    model.fit(X_train, y_train)

    # Realizar predicciones
    y_pred = model.predict(X_test)

    # Calcular y mostrar la precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    matriz_conf = confusion_matrix(y_test, y_pred)
    reporte = classification_report(y_test, y_pred)
    
    dicc_forest[model.n_estimators] = matriz_conf
    dicc_reporte[model.n_estimators] = reporte
    print(i)

0
1
2
3
4


In [35]:
print(dicc_forest[20])

[[  4191    347   1542     40   5042]
 [  1180    523   2416    115   4180]
 [   753    383   6935   2178  19908]
 [   307     57   4537   8474  50605]
 [   396     28   1592   6319 109011]]


In [36]:
print(dicc_forest[100])

[[  4191    348   1500     87   5036]
 [  1166    534   2361    179   4174]
 [   750    384   6722   2519  19782]
 [   308     57   4367   9147  50101]
 [   394     29   1449   6938 108536]]


In [11]:
for j in n_arboles:
    precision = np.diagonal(dicc_forest[j]).sum() / dicc_forest[j].sum()
    print("Cantidad de arboles:",j, "-> Precision:", precision)

Cantidad de arboles: 50 -> Precision: 0.6013578841071697
Cantidad de arboles: 100 -> Precision: 0.6014796353193105
Cantidad de arboles: 5 -> Precision: 0.600183343001812
Cantidad de arboles: 20 -> Precision: 0.6013722077791863
Cantidad de arboles: 10 -> Precision: 0.6005342729662176


In [12]:
print(dicc_reporte[100])

              precision    recall  f1-score   support

         1.0       0.59      0.60      0.60      6757
         2.0       0.36      0.10      0.15      5105
         3.0       0.42      0.41      0.42     16727
         4.0       0.50      0.23      0.32     39134
         5.0       0.66      0.88      0.75     71906

    accuracy                           0.60    139629
   macro avg       0.51      0.44      0.45    139629
weighted avg       0.57      0.60      0.56    139629



In [10]:
df = df_final.head(100000)

In [11]:
from sklearn.model_selection import train_test_split

X, y = df.drop('score',axis=1),df['score']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [12]:
from sklearn import svm

#Cargamos también las librerías para hacer la matriz de confusión y la cross validation:

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold


In [13]:
from sklearn.svm import SVC

C = {0.1, 1, 10, 100, 1000}
kernel = 'rbf'
gamma = 'scale'

svc = [SVC(C=i, kernel=kernel, gamma=gamma) for i in C]
svc

[SVC(C=0.1), SVC(C=1), SVC(C=100), SVC(C=1000), SVC(C=10)]

In [14]:
dicc_svc = {}
dicc_reporte_svc = {}

for i, model in enumerate(svc):
    # Entrenar el modelo con datos de entrenamiento
    model.fit(X_train, y_train)

    # Realizar predicciones
    y_pred = model.predict(X_test)

    # Calcular y mostrar la precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    matriz_conf = confusion_matrix(y_test, y_pred)
    reporte = classification_report(y_test, y_pred)
    
    dicc_svc[model.C] = matriz_conf
    dicc_reporte_svc[model.C] = reporte
    print(i)

D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

0


D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

1


D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

2
3
4


D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Usuarios\jchata\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [15]:
for k in C:
    print("Modelo C = ", k)
    cnt_correcto = np.diagonal(dicc_svc[k]).sum()
    print(dicc_svc[k])
    print("Cantidad clasificados correctamente:", cnt_correcto)

Modelo C =  0.1
[[ 676    0  152   40   92]
 [ 307    0  200   97  157]
 [ 325    0  288  488 1393]
 [ 137    0  101  409 4969]
 [ 196    0   49  198 9726]]
Cantidad clasificados correctamente: 11099
Modelo C =  1
[[ 613    0  215   65   67]
 [ 218    0  289  155   99]
 [ 178    0  435  801 1080]
 [  97    0  141  846 4532]
 [ 172    0   73  474 9450]]
Cantidad clasificados correctamente: 11344
Modelo C =  100
[[ 613    0  215   90   42]
 [ 214    0  293  210   44]
 [ 176    0  437 1449  432]
 [  96    0  142 2853 2525]
 [ 172    0   73 2381 7543]]
Cantidad clasificados correctamente: 11446
Modelo C =  1000
[[ 613    0  215   90   42]
 [ 214    2  290  210   45]
 [ 176    0  437 1449  432]
 [  96    0  142 2853 2525]
 [ 172    0   73 2381 7543]]
Cantidad clasificados correctamente: 11448
Modelo C =  10
[[ 613    0  215   90   42]
 [ 214    0  293  210   44]
 [ 176    0  437 1446  435]
 [  96    0  142 2852 2526]
 [ 172    0   73 2378 7546]]
Cantidad clasificados correctamente: 11448


In [16]:
for k in C:
    precision = np.diagonal(dicc_svc[k]).sum() / dicc_svc[k].sum()
    print("Modelo C:",k , "-> Precision:", precision)
    print(dicc_reporte_svc[k])

Modelo C: 0.1 -> Precision: 0.55495
              precision    recall  f1-score   support

         1.0       0.41      0.70      0.52       960
         2.0       0.00      0.00      0.00       761
         3.0       0.36      0.12      0.18      2494
         4.0       0.33      0.07      0.12      5616
         5.0       0.60      0.96      0.73     10169

    accuracy                           0.55     20000
   macro avg       0.34      0.37      0.31     20000
weighted avg       0.46      0.55      0.45     20000

Modelo C: 1 -> Precision: 0.5672
              precision    recall  f1-score   support

         1.0       0.48      0.64      0.55       960
         2.0       0.00      0.00      0.00       761
         3.0       0.38      0.17      0.24      2494
         4.0       0.36      0.15      0.21      5616
         5.0       0.62      0.93      0.74     10169

    accuracy                           0.57     20000
   macro avg       0.37      0.38      0.35     20000
weighted